In [1]:
!pip install opencv-python dlib scikit-image tensorflow


In [4]:
import cv2
import dlib
import os
import zipfile
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [5]:
zip_path = '/content/archive.zip'

extract_path = '/content/makeup/'

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify the contents of the extracted folder
extracted_files = os.listdir(extract_path)
print(extracted_files)

['make_up.csv', 'with_makeup', 'no_makeup']


In [6]:
def apply_lipstick(image, lips):
    lip_mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(lip_mask, [np.array(lips, dtype=np.int32)], 255)
    image_with_lipstick = image.copy()
    image_with_lipstick[lip_mask == 255] = (0, 0, 255)  # Red for lipstick
    return image_with_lipstick

def apply_eyeshadow(image, eyes):
    eye_mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(eye_mask, [np.array(eyes, dtype=np.int32)], 255)
    image_with_eyeshadow = image.copy()
    image_with_eyeshadow[eye_mask == 255] = (255, 0, 0)  # Blue for eyeshadow
    return image_with_eyeshadow

def apply_blush(image, blush_points):
    image_with_blush = image.copy()
    for point in blush_points:
        cv2.circle(image_with_blush, point, 20, (147, 112, 219), -1)  # Purple for blush
    return image_with_blush

def apply_virtual_makeup(image, landmarks):
    lips = landmarks[48:60]
    eyes = landmarks[36:48]
    blush = [landmarks[i] for i in [1, 2, 3, 4, 5, 6]]  # Simple blush points
    image_with_lipstick = apply_lipstick(image, lips)
    image_with_eyeshadow = apply_eyeshadow(image_with_lipstick, eyes)
    image_with_blush = apply_blush(image_with_eyeshadow, blush)
    return image_with_blush


In [7]:
# Initialize dlib face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def get_face_regions_dlib(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        return None
    shape = predictor(gray, faces[0])
    landmarks = [(shape.part(i).x, shape.part(i).y) for i in range(68)]
    return landmarks


In [8]:
def process_images_for_makeup_dlib(no_makeup_folder, output_folder):
    image_files = [f for f in os.listdir(no_makeup_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for image_file in image_files:
        image_path = os.path.join(no_makeup_folder, image_file)
        image = cv2.imread(image_path)
        landmarks = get_face_regions_dlib(image)
        if landmarks:
            processed_image = apply_virtual_makeup(image, landmarks)
            cv2.imwrite(os.path.join(output_folder, image_file), processed_image)


In [9]:
def compare_images(image1, image2):
    if image1.shape != image2.shape:
        image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]))

    gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    score, _ = ssim(gray_image1, gray_image2, full=True)
    return score

def calculate_overall_ssim(processed_images_folder, with_makeup_folder):
    processed_images = os.listdir(processed_images_folder)
    with_makeup_images = os.listdir(with_makeup_folder)

    total_ssim_score = 0
    valid_comparisons = 0

    for image_file in processed_images:
        if image_file in with_makeup_images:
            processed_image_path = os.path.join(processed_images_folder, image_file)
            processed_image = cv2.imread(processed_image_path)

            real_makeup_image_path = os.path.join(with_makeup_folder, image_file)
            real_makeup_image = cv2.imread(real_makeup_image_path)

            ssim_score = compare_images(processed_image, real_makeup_image)
            total_ssim_score += ssim_score
            valid_comparisons += 1

    if valid_comparisons > 0:
        return total_ssim_score / valid_comparisons
    else:
        return 0


In [10]:
def calculate_metrics_with_threshold(processed_images_folder, with_makeup_folder, threshold=0.5):
    processed_images = os.listdir(processed_images_folder)
    with_makeup_images = os.listdir(with_makeup_folder)

    y_true = []  # Ground-truth labels (1 for success, 0 for failure)
    y_pred = []  # Predicted labels (based on SSIM threshold)

    for image_file in processed_images:
        if image_file in with_makeup_images:
            processed_image_path = os.path.join(processed_images_folder, image_file)
            processed_image = cv2.imread(processed_image_path)

            real_makeup_image_path = os.path.join(with_makeup_folder, image_file)
            real_makeup_image = cv2.imread(real_makeup_image_path)

            ssim_score = compare_images(processed_image, real_makeup_image)

            y_true.append(1)
            y_pred.append(1 if ssim_score >= threshold else 0)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred, zero_division=1)
    f1 = f1_score(y_true, y_pred, zero_division=1)

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }


In [11]:
# Paths to folders
no_makeup_folder = '/content/makeup/no_makeup/'
with_makeup_folder = '/content/makeup/with_makeup/'
output_folder = '/content/output_dlib/'

# Process images
process_images_for_makeup_dlib(no_makeup_folder, output_folder)

# Calculate SSIM score
ssim_score = calculate_overall_ssim(output_folder, with_makeup_folder)
print(f"SSIM Score: {ssim_score:.4f}")


metrics = calculate_metrics_with_threshold(output_folder, with_makeup_folder, threshold=0.5)
print("\nClassification Metrics:")
print(f"Accuracy: {metrics['Accuracy']:.4f}")
print(f"Precision: {metrics['Precision']:.4f}")
print(f"Recall: {metrics['Recall']:.4f}")
print(f"F1-Score: {metrics['F1-Score']:.4f}")


SSIM Score: 0.5197

Classification Metrics:
Accuracy: 0.6400
Precision: 1.0000
Recall: 0.6400
F1-Score: 0.7805
